# Análisis de datos de la pandemia de covid-19 en la Ciudad de México

En este proyecto, presento algunos análisis y visualizaciones de datos sobre la pandemia de Covid-19 en la Ciudad de México. Me concentro únicamente (por facilidad) en datos sobre personas con una prueba de laboratorio positiva (no antígeno).

La base de datos es la de la Secretaría de Salud y puede ser consultada en su portal. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
import covid_data # Código propio para cargar y leer la base de datos de la SSA

In [3]:
fecha = "210907"
covid_data.descarga_base_covid(fecha)

200
Descarga exitosa
La descarga tomó 147.84022092819214 segundos


In [4]:
datos = covid_data.lee_base_covid(fecha)

C:\Users\berto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


La base de datos tardó en cargarse 30.825266361236572 segundos


In [5]:
datos.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-09-07,z482b8,2,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,México,97,97
1,2021-09-07,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
2,2021-09-07,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,México,97,97
3,2021-09-07,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,México,97,97
4,2021-09-07,zz8e77,2,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,México,97,97


In [6]:
datos.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

Selecciono solo algunas columnas del conjunto de datos para estudiarlas. Entre estas columnas, selecciono las asociadas a fechas y de comorbilidades.

In [7]:
columnas = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'SEXO', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'EMBARAZO','DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', "RESULTADO_LAB"]
datos_ = datos[columnas]

In [29]:
datos_covid = datos_[datos_['ENTIDAD_RES']==9] #CIUDAD DE MÉXICO
datos_covid = datos_covid[datos_covid["RESULTADO_LAB"]==1] # PRUEBA PCR POSITIVA

In [10]:
datos_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320038 entries, 15 to 10005697
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ENTIDAD_RES     320038 non-null  int64 
 1   MUNICIPIO_RES   320038 non-null  int64 
 2   SEXO            320038 non-null  int64 
 3   TIPO_PACIENTE   320038 non-null  int64 
 4   FECHA_INGRESO   320038 non-null  object
 5   FECHA_SINTOMAS  320038 non-null  object
 6   FECHA_DEF       320038 non-null  object
 7   INTUBADO        320038 non-null  int64 
 8   NEUMONIA        320038 non-null  int64 
 9   EDAD            320038 non-null  int64 
 10  EMBARAZO        320038 non-null  int64 
 11  DIABETES        320038 non-null  int64 
 12  EPOC            320038 non-null  int64 
 13  ASMA            320038 non-null  int64 
 14  INMUSUPR        320038 non-null  int64 
 15  HIPERTENSION    320038 non-null  int64 
 16  OTRA_COM        320038 non-null  int64 
 17  CARDIOVASCULAR  320038 non

## Limpieza de datos

Primero realizaré limpieza de datos. Convertiré algunas columnas a tipos más sencillos con los que trabajar y añadiré unas cuantas columnas extras que serán de utilidad.

In [30]:
datos_covid["FECHA_INGRESO"] = pd.to_datetime(datos_covid["FECHA_INGRESO"])
datos_covid["FECHA_SINTOMAS"] = pd.to_datetime(datos_covid["FECHA_SINTOMAS"])

La columna "FECHA_DEF" tiene entradas nulas que están escritas en el formato "9999-99-99". Convertiré estas entradas a unas que la función pd.to_datetime entienda y cuando no las necesite en análisis posteriores, simplemente omitiré estas entradas.

In [22]:
def fecha_nula(fecha):
    """
    Una función para checar fechas nulas en la columna de fecha de defunción
    """
    if fecha == "9999-99-99":
        fecha = "1999-01-01"
    return fecha

In [32]:
def defuncion(fecha):
    if fecha == "1999-01-01":
        d = 0
    else:
        d = 1
    return d

In [33]:
datos_covid["DEF"] = datos_covid["FECHA_DEF"].apply(defuncion)

In [34]:
datos_covid["FECHA_DEF"] = datos_covid["FECHA_DEF"].apply(fecha_nula)

In [35]:
datos_covid["FECHA_DEF"] = pd.to_datetime(datos_covid["FECHA_DEF"])

In [25]:
datos_covid["SINTOMAS_INGRESO"] = datos_covid.FECHA_INGRESO - datos_covid.FECHA_SINTOMAS
datos_covid["SINTOMAS_DEF"] = datos_covid.FECHA_DEF - datos_covid.FECHA_SINTOMAS

In [37]:
datos_covid.reset_index(inplace=True, drop=True)

In [38]:
datos_covid.head()

,ENTIDAD_RES,MUNICIPIO_RES,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,...,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,RESULTADO_LAB,DEF
0,9,6,1,1,2020-09-08,2020-09-03,1999-01-01,97,2,26,...,2,2,2,2,2,2,2,2,1,1
1,9,5,1,1,2020-04-22,2020-04-20,1999-01-01,97,2,45,...,2,2,2,2,2,1,2,2,1,1
2,9,4,1,1,2020-09-23,2020-09-23,1999-01-01,97,2,67,...,2,2,2,2,2,2,2,2,1,1
3,9,3,1,1,2020-03-04,2020-03-03,1999-01-01,97,2,29,...,2,2,2,2,2,2,2,2,1,1
4,9,5,1,2,2020-03-12,2020-03-11,1999-01-01,2,2,25,...,2,2,2,2,2,2,2,2,1,1
